In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
from torch.utils.data import DataLoader, TensorDataset
from data_processing import generate_dataset_from_pgn, label_to_move_table, fen_to_board
import chess
import random

In [7]:


os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

dataset = generate_dataset_from_pgn("masaurus101-white.pgn")
train_to_test_ratio = 0.8

train_size = int(len(dataset) * train_to_test_ratio)
test_size = len(dataset) - train_size

# Split the dataset
train_data = dataset[:train_size]
test_data = dataset[train_size:]

# Convert to tensors (simpler now since labels are already integers!)
X_train = torch.stack([board for board, label in train_data])  # (N, 8, 8, 12)
t_train = torch.tensor([label for board, label in train_data])  # (N,)

X_test = torch.stack([board for board, label in test_data])
t_test = torch.tensor([label for board, label in test_data])

# Create DataLoaders
batch_size = 32
train_dataset = TensorDataset(X_train, t_train)
test_dataset = TensorDataset(X_test, t_test)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [8]:

class SLPolicyNetwork(nn.Module):
    def __init__(self, num_possible_moves=20480):
        super(SLPolicyNetwork, self).__init__()

        self.conv1 = nn.Conv2d(
            in_channels=12, out_channels=32, kernel_size=3, padding=1
        )
        self.conv2 = nn.Conv2d(
            in_channels=32, out_channels=64, kernel_size=3, padding=1
        )
        self.conv3 = nn.Conv2d(
            in_channels=64, out_channels=128, kernel_size=3, padding=1
        )

        self.fc1 = nn.Linear(128 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, num_possible_moves)

    def forward(self, x):
        x = x.permute(0, 3, 1, 2)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))

        x = torch.flatten(x, start_dim=1)  # exclude batch dimension
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x


model = SLPolicyNetwork()
model.load_state_dict(torch.load("sl_policy_network_KC.pth", map_location=torch.device("cpu")))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.1e-4)

In [9]:

def predict_move(model, board_tensor):
    """
    Takes a board tensor (8, 8, 12) and returns the predicted UCI move.
    """
    label_to_uci = label_to_move_table()
    model.eval()  # Set to evaluation mode
    

    with torch.no_grad():  # No gradients needed for inference
        # Add batch dimension: (8, 8, 12) -> (1, 8, 8, 12)
        board_batch = board_tensor.unsqueeze(0)

        # Get model output
        logits = model(board_batch)  # Shape: (1, 20480)
        probabilities = F.softmax(logits, dim=1)

        # Get the highest scoring move
        predicted_label = torch.argmax(probabilities, dim=1).item()

        # Convert to UCI
        predicted_uci = label_to_uci[predicted_label]

    return predicted_uci, probabilities[0][predicted_label]


def list_predicted_moves(model, board_tensor, num_moves):
    label_to_uci = label_to_move_table()

    model.eval()
    with torch.no_grad():
        board_batch = board_tensor.unsqueeze(0)
        logits = model(board_batch)  
        probabilities = F.softmax(logits, dim=1)
        score, moves = torch.topk(probabilities, num_moves)
        moves = [label_to_uci[int(move)] for move in moves[0]]
        

    return moves, score



In [10]:
# epochs = 1

# for epoch in range(epochs):
#     for batch_idx, (data, target) in enumerate(train_dataloader):
#         output = model(data)  # calculate predictions for this batch
#         loss = criterion(output, target)  # calculate loss
#         optimizer.zero_grad()  # reset gradient
#         loss.backward()  # calculate gradient
#         optimizer.step()  # update parameters

#         if batch_idx % 100 == 0:
#             print(f"Epoch {epoch+1}: Loss = {loss.item():.4f}")

    # doesn't really make sense to calculate validation accuracy as opening move has many possible moves

    # model.eval()
    # test_loss = 0
    # correct = 0

    # with torch.no_grad():
    #     for data, target in test_dataloader:
    #         # data, target = data.to(device), target.to(device)
    #         output = model(data)
    #         test_loss += criterion(output, target).item()
    #         correct += (output.argmax(1) == target).type(torch.float).sum().item()

    # print('epoch: {}, test loss: {:.6f}, test accuracy: {:.6f}'.format(
    #     epoch + 1,
    #     test_loss / len(test_dataloader),
    #     correct / len(test_dataloader.dataset)
    #     ))

In [11]:
# import chess
# board = chess.Board()
# board.push_uci('d2d4')
# board_tensor = fen_to_board(board.fen())
# print(predict_move(model, board_tensor))
# print(list_predicted_moves(model, board_tensor, 5))

# label_to_move_table()

# MODEL PREDICTS ILLEGAL MOVES

In [21]:
board = chess.Board()
moves_played = []
model_turn = 1

while not board.is_game_over():
    move = None
    if model_turn:
        print("model")

        board_tensor = fen_to_board(board.fen())
        moves, probs = list_predicted_moves(model, board_tensor, 20480)
        
        for move in moves:
            try:
                board.push_uci(move)
            except chess.IllegalMoveError:
                continue
            break
        model_turn = 0
    else:
        print("random")
        moves = board.legal_moves
        move_index = random.randint(0, moves.count()-1)
        moves = [move for move in moves]
        move = moves[move_index]
        board.push(move)
        model_turn = 1

    moves_played.append(move)
    print(board)
    print(moves_played)
    

model
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R
['e2e4']
random
r n b q k b n r
p . p p p p p p
. . . . . . . .
. p . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R
['e2e4', Move.from_uci('b7b5')]
model
r n b q k b n r
p . p p p p p p
. . . . . . . .
. p . . . . . .
. . . P P . . .
. . . . . . . .
P P P . . P P P
R N B Q K B N R
['e2e4', Move.from_uci('b7b5'), 'd2d4']
random
r n b q k b n r
p . p . p p p p
. . . p . . . .
. p . . . . . .
. . . P P . . .
. . . . . . . .
P P P . . P P P
R N B Q K B N R
['e2e4', Move.from_uci('b7b5'), 'd2d4', Move.from_uci('d7d6')]
model
r n b q k b n r
p . p . p p p p
. . . p . . . .
. p . . . . . .
. . . P P . . .
. . N . . . . .
P P P . . P P P
R . B Q K B N R
['e2e4', Move.from_uci('b7b5'), 'd2d4', Move.from_uci('d7d6'), 'b1c3']
random
r n b . k b n r
p . p q p p p p
. . . p . . . .
. p . . . . . .
. . . P P . . .
. . N . . . . .
P P P . . 

In [18]:

def export_game_from_board(board: chess.Board, file_name: str):
    game = chess.pgn.Game()
    game.headers["Event"] = "AI Self Play"
    game.headers["White"] = "Your Model"
    game.headers["Black"] = "Your Model"
    game.headers["Result"] = board.result()

    # add moves to the game node
    node = game
    for move in board.move_stack:
        node = node.add_variation(move)

    # save to PGN file
    with open(f"{file_name}.pgn", "w", encoding="utf-8") as pgn_file:
        print(game, file=pgn_file)
    pgn_file.close()


20


In [22]:
export_game_from_board(board, "model_vs_random")